# **FACTORY METHOD**

_Factory Method is a creational design pattern that provides an interface for creating objects in a superclass, but allows subclasses to alter the type of objects that will be created._




In [ ]:
# SOURCE: https://refactoring.guru/design-patterns/factory-method/python/example#lang-features

from __future__ import annotations

from abc import ABC, abstractmethod


class Creator(ABC):
    """
    The Creator class declares the factory method that is supposed to return an
    object of a Product class. The Creator's subclasses usually provide the
    implementation of this method.
    """

    @abstractmethod
    def factory_method(self) -> Product:
        """
        Note that the Creator may also provide some default implementation of
        the factory method.
        """
        pass

    def some_operation(self) -> str:
        """
        Also note that, despite its name, the Creator's primary responsibility
        is not creating products. Usually, it contains some core business logic
        that relies on Product objects, returned by the factory method.
        Subclasses can indirectly change that business logic by overriding the
        factory method and returning a different type of product from it.
        """

        # Call the factory method to create a Product object.
        product = self.factory_method()

        # Now, use the product.
        result = f"Creator: The same creator's code has just worked with {product.operation()}"

        return result


"""
Concrete Creators override the factory method in order to change the resulting
product's type.
"""


class ConcreteCreator1(Creator):
    """
    Note that the signature of the method still uses the abstract product type,
    even though the concrete product is actually returned from the method. This
    way the Creator can stay independent of concrete product classes.
    """

    def factory_method(self) -> Product:
        return ConcreteProduct1()


class ConcreteCreator2(Creator):
    def factory_method(self) -> Product:
        return ConcreteProduct2()


class Product(ABC):
    """
    The Product interface declares the operations that all concrete products
    must implement.
    """

    @abstractmethod
    def operation(self) -> str:
        pass


"""
Concrete Products provide various implementations of the Product interface.
"""


class ConcreteProduct1(Product):
    def operation(self) -> str:
        return "{Result of the ConcreteProduct1}"


class ConcreteProduct2(Product):
    def operation(self) -> str:
        return "{Result of the ConcreteProduct2}"


def client_code(creator: Creator) -> None:
    """
    The client code works with an instance of a concrete creator, albeit through
    its base interface. As long as the client keeps working with the creator via
    the base interface, you can pass it any creator's subclass.
    """

    print(
        f"Client: I'm not aware of the creator's class, but it still works.\n"
        f"{creator.some_operation()}",
        end="",
    )


print("App: Launched with the ConcreteCreator1.")
client_code(ConcreteCreator1())
print("\n")

print("App: Launched with the ConcreteCreator2.")
client_code(ConcreteCreator2())

App: Launched with the ConcreteCreator1.
Client: I'm not aware of the creator's class, but it still works.
Creator: The same creator's code has just worked with {Result of the ConcreteProduct1}

App: Launched with the ConcreteCreator2.
Client: I'm not aware of the creator's class, but it still works.
Creator: The same creator's code has just worked with {Result of the ConcreteProduct2}

### **CONCRETE EXAMPLES**

Imagine an application that needs to convert a Song object into its string representation using a specified format. Converting an object to a different representation is often called serializing. You’ll often see these requirements implemented in a single function or method that contains all the logic and implementation, like in the following code:

In [2]:
import json
import xml.etree.ElementTree as et


class Song:
    def __init__(self, song_id, title, artist) -> None:
        self.song_id = song_id
        self.title = title
        self.artist = artist


class SongSerializer:
    def serialize(self, song, format) -> str:
        if format == "JSON":
            song_info = {
                "id": song.song_id,
                "title": song.title,
                "artist": song.artist,
            }
            return json.dumps(song_info)
        elif format == "XML":
            song_info = et.Element("song", attrib={"id": song.song_id})
            title = et.SubElement(song_info, "title")
            title.text = song.title
            artist = et.SubElement(song_info, "artist")
            artist.text = song.artist
            return et.tostring(song_info, encoding="unicode")
        else:
            raise ValueError(format)

In [3]:
song = Song("1", "Water of Love", "Dire Straits")
serializer = SongSerializer()

print(serializer.serialize(song, "JSON"))
print(serializer.serialize(song, "XML"))

{"id": "1", "title": "Water of Love", "artist": "Dire Straits"}
<song id="1"><title>Water of Love</title><artist>Dire Straits</artist></song>


The code above is hard to maintain because it is doing too much. The single responsibility principle states that a module, a class, or even a method should have a single, well-defined responsibility. It should do just one thing and have only one reason to change.

The `serialize()` method in `SongSerializer` will require changes for many different reasons. This increases the risk of introducing new defects or breaking existing functionality when changes are made.

In [4]:
import json
import xml.etree.ElementTree as et
from typing import Any, Protocol
from xml.etree.ElementTree import Element

import yaml


class Serializer(Protocol):
    def start_object(self, object_name: str, object_id: str) -> None: ...
    def add_property(self, name: str, value: Any) -> None: ...
    def to_str(self) -> str: ...


class JsonSerializer(Serializer):
    _current_object: dict[str, Any]

    def start_object(self, object_name: str, object_id: str) -> None:
        self._current_object = {"id": object_id}

    def add_property(self, name: str, value: Any) -> None:
        self._current_object[name] = value

    def to_str(self) -> str:
        return json.dumps(self._current_object)


class XmlSerializer(Serializer):
    _element: Element

    def start_object(self, object_name: str, object_id: str) -> None:
        self._element = et.Element(object_name, attrib={"id": object_id})

    def add_property(self, name: str, value: Any) -> None:
        prop = et.SubElement(self._element, name)
        prop.text = value

    def to_str(self) -> str:
        return et.tostring(self._element, encoding="unicode")


class YamlSerializer(JsonSerializer):
    def to_str(self) -> str:
        return yaml.dump(self._current_object)


class SerializerFactory:
    def __init__(self) -> None:
        self._serializers: dict[str, Serializer] = dict()

    def register_format(self, format: str, serializer: Serializer) -> None:
        self._serializers[format] = serializer

    def get_serializer(self, format) -> Serializer:
        serializer = self._serializers.get(format)
        if serializer is None:
            raise ValueError(f"Unknown format: {format}")
        return serializer


factory = SerializerFactory()
factory.register_format("JSON", JsonSerializer())
factory.register_format("XML", XmlSerializer())
factory.register_format("YAML", YamlSerializer())


class Serializable(Protocol):
    def serialize(self, serializer: Serializer) -> None: ...


class ObjectSerializer:
    def serialize(self, serializable: Serializable, format: str) -> str:
        serializer = factory.get_serializer(format)
        serializable.serialize(serializer)
        return serializer.to_str()


class Song:
    def __init__(self, song_id: str, title: str, artist: str) -> None:
        self.song_id = song_id
        self.title = title
        self.artist = artist

    def serialize(self, serializer: Serializer) -> None:
        serializer.start_object("song", self.song_id)
        serializer.add_property("title", self.title)
        serializer.add_property("artist", self.artist)


song = Song("1", "Water of Love", "Dire Straits")
serializer = ObjectSerializer()

print(serializer.serialize(song, "JSON"))
print(serializer.serialize(song, "XML"))
print(serializer.serialize(song, "YAML"))

{"id": "1", "title": "Water of Love", "artist": "Dire Straits"}
<song id="1"><title>Water of Love</title><artist>Dire Straits</artist></song>
artist: Dire Straits
id: '1'
title: Water of Love

